In [137]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import scipy
import math
# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [62]:
df=pd.read_excel(io='table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls')

In [63]:
df=df.drop(df.index[:4])

In [64]:
df=df.drop(df.index[348:])

In [65]:
df.columns=['City','Population','Violent crime','Murder and nonnegligent manslaughter','Rape (revised definition)','Rape (legacy definition)','Robbery','Aggravated assault','Property crime','Burglary','Larceny- theft','Motor vehicle theft','Arson']

In [66]:
display(df.head())

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition),Rape (legacy definition),Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
4,Adams Village,1861,0,0,NaN,0,0,0,12,2,10,0,0
5,Addison Town and Village,2577,3,0,NaN,0,0,3,24,3,20,1,0
6,Akron Village,2846,3,0,NaN,0,0,3,16,1,15,0,0
7,Albany,97956,791,8,NaN,30,227,526,4090,705,3243,142,NaN
8,Albion Village,6388,23,0,NaN,3,4,16,223,53,165,5,NaN


In [97]:
df.isnull().sum()

City                                      0
Population                                0
Violent crime                             0
Murder and nonnegligent manslaughter      0
Rape (revised definition)               348
Rape (legacy definition)                  0
Robbery                                   0
Aggravated assault                        0
Property crime                            0
Burglary                                  0
Larceny- theft                            0
Motor vehicle theft                       0
Arson                                   161
dtype: int64

In [67]:
features = pd.DataFrame({'A':[]})
features_2 = pd.DataFrame({'A':[]})

In [68]:
features['population']=df['Population']
features_2['population']=df['Population']

In [69]:
features['pop_squared']=df['Population']**2
features_2['pop_squared']=df['Population']**2

In [70]:
features['murder'] = df['Murder and nonnegligent manslaughter']/df['Population']
features_2['murder'] = df['Murder and nonnegligent manslaughter']

In [71]:
features['robbery'] = df['Robbery']/df['Population']
features_2['robbery'] = df['Robbery']

In [72]:
features = features.drop('A',axis=1)
features_2 = features_2.drop('A',axis=1)

In [73]:
features['larceny']= df['Larceny- theft']/df['Population']
features_2['larceny']= df['Larceny- theft']

In [74]:
features['burglary']= df['Burglary']/df['Population']
features_2['burglary']= df['Burglary']

In [102]:
features['property_crime']= df['Property crime']/df['Population']
features_2['property_crime']= df['Property crime']/df['Population']

In [125]:
features.head()

,population,pop_squared,murder,robbery,larceny,burglary,property crime,property_crime
4,1861,3463321,0.000,0.000,0.005,0.001,0.006,0.006
5,2577,6640929,0.000,0.000,0.008,0.001,0.009,0.009
6,2846,8099716,0.000,0.000,0.005,0.000,0.006,0.006
7,97956,9595377936,0.000,0.002,0.033,0.007,0.042,0.042
8,6388,40806544,0.000,0.001,0.026,0.008,0.035,0.035


In [123]:
features_2.head()

,population,pop_squared,murder,robbery,larceny,burglary,property crime,property_crime
4,1861,3463321,0,0,10,2,12,0.006
5,2577,6640929,0,0,20,3,24,0.009
6,2846,8099716,0,0,15,1,16,0.006
7,97956,9595377936,8,227,3243,705,4090,0.042
8,6388,40806544,0,4,165,53,223,0.035


In [121]:
#features_2[features_2['population']<8000000]['population'].hist(bins=10)

In [114]:
features_2.sort_values('population', ascending=False)

,population,pop_squared,murder,robbery,larceny,burglary,property crime,property_crime
220,8396126,70494931807876,335,19170,117931,16606,141971,0.017
39,258789,66971746521,47,1322,8076,3458,12491,0.048
276,210562,44336355844,42,918,6855,2587,10051,0.048
350,199134,39654349956,6,390,1662,470,2368,0.012
314,143834,20688219556,21,400,4298,1781,6473,0.045
11,118296,13993943616,1,31,1882,204,2118,0.018
7,97956,9595377936,8,227,3243,705,4090,0.042
130,96667,9344508889,0,60,1925,332,2303,0.024
271,87204,7604537616,2,9,466,88,567,0.007
61,80705,6513297025,0,17,1388,99,1515,0.019


In [79]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y= features['property crime']#.apply(lambda x: np.log(float(x)))
# per capita
X = features[['population','murder','robbery','larceny','burglary']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [-2.40493710e-11  8.97961915e-01  3.72168065e-01  1.00743899e+00
  1.03505076e+00]

Intercept: 
 8.313580240868965e-05

R-squared:
0.9991184272078656


In [105]:
### Using this model moving forward ###
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y= features_2['property_crime']#.apply(lambda x: np.log(float(x)))
X = features_2[['population','murder','robbery','larceny','burglary']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [-5.12651766e-07 -2.80108264e-03  5.33564065e-05  3.87157353e-05
 -2.09233055e-05]

Intercept: 
 0.01834489412705064

R-squared:
0.32136223673417497


In [127]:
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y= features_2['property crime']*features_2['population']#.apply(lambda x: np.log(float(x)))
X = features_2[['population','murder','robbery','larceny','burglary']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [ 3.79278512e+03 -2.72404721e+08  6.34243388e+07  4.27105668e+06
 -2.82230071e+07]

Intercept: 
 -235766921.4506564

R-squared:
0.9993521645395048


In [142]:
# Use a ~ to represent an '=' from the functional form
linear_formula = 'property_crime ~ population+murder+robbery+larceny+burglary'

# Fit the model to our data using the formula.
lm = smf.ols(formula=linear_formula, data=features.reset_index()).fit()

In [148]:
features.apply(float).dtypes

TypeError: ("cannot convert the series to <class 'float'>", 'occurred at index population')

In [144]:
lm.summary()

ValueError: shapes (348,342) and (348,342) not aligned: 342 (dim 1) != 348 (dim 0)

In [130]:
lm.pvalues

ValueError: shapes (348,342) and (348,342) not aligned: 342 (dim 1) != 348 (dim 0)

In [109]:
lm.rsquared

ValueError: shapes (348,342) and (348,342) not aligned: 342 (dim 1) != 348 (dim 0)